In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import numpy as np

ubuntu = True

if not ubuntu:
    plt.rcParams['animation.ffmpeg_path'] = 'C:\\Program Files\\ImageMagick-7.0.7-Q16\\ffmpeg.exe'
    plt.rcParams["animation.convert_path"] = "C:\\Program Files\\ImageMagick-7.0.7-Q16\\magick.exe"
else:
    from matplotlib.animation import ImageMagickFileWriter
    writer = ImageMagickFileWriter()
    plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
    plt.rcParams["animation.convert_path"] = "/usr/bin/convert"

% matplotlib inline

In [2]:
i = 9999

Tf = np.genfromtxt('out/output_' + str(i) + '.csv')
Tf = Tf.transpose()

X = np.arange(Tf.shape[1])
Y = np.arange(Tf.shape[0])

In [3]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(12,15))

im = ax1.pcolormesh(X,Y,Tf, label='Temperature en Kelvin')
ax1.set_title("Champ de température final")
fig.colorbar(im, ax=ax1)
ax1.set_xlabel("x")
ax1.set_ylabel("y")

yfix = [0,1,3,5,7,9]
for y in yfix:
    ax2.plot(X,Tf[y,:], label='y={}'.format(y))
ax2.set_title("Température en fonction de x pour y fixé")
ax2.set_xlabel("x")
ax2.set_ylabel("T(K)")
ax2.legend()

In [7]:
# Comparaison avec la solution analytique

analytique = True

if analytique:
    from analy import analy
    from variables import T1, Ta, h, lamb, Lx, Ly, Nx, Ny
    
    n = 100
    err = 0.0001
    Tanaly = analy(T1, Ta, h, lamb, Lx, Ly, Nx, Ny, n=n, err=err)
    
    

In [8]:
imax = 10000
T = []
for i in range(0,imax,5):
    T.append(np.genfromtxt('out/output_' + str(i+1) + '.csv'))

for i in range(len(T)):
    T[i] = T[i].transpose()
    if analytique:
        T[i] = T[i] - Tanaly

In [9]:
fig, ax = plt.subplots()
xdata, ydata, Tdata = [], [], []

im = ax.pcolormesh(X,Y,T[0], label='Temperature en Kelvin')

fig.colorbar(im, ax=ax)

def init():
    if not analytique:
        ax.set_title("Champ de température")
    else:
        ax.set_title("Erreur de simulation")
    ax.set_xlabel("x(m)")
    ax.set_ylabel("y(m)")
    return im,

def update(frame):
#     xdata = X
#     ydata = Y
#     Tdata = T[frame]
    im = ax.pcolormesh(X,Y,T[frame], label='Temperature en Kelvin')
    return im,

skipping = 10 # default 1
gifname = "long_error.gif"
ani = FuncAnimation(fig, update, frames=range(1,len(T), skipping), init_func=init, blit=False)
if not ubuntu:
    ani.save(gifname, writer="imagemagick")
else:
    ani.save(gifname, writer=writer)